# 1 Introducción
El siguiente cuaderno realiza el calculo del tiempo que tarda en caer un objeto desde una altura h, utilizando el procesador GPU. El algoritmo se basa en la ecuacion de tiempo de "Caida Libre" para calcular el tiempo de caida [5].

<center>$t = √(2h/g)$</center>
<center>t = tiempo en caer;  h = altura inicial; g = gravedad</center>

El objetivo es realizar el calculo del tiempo que tarda en caer un objeto desde una altura h.

---
# 2 Armado del ambiente
Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

---
# 3 Desarrollo
Ejecuta el Código CPU - GPU.

In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }
#@markdown ---
#@markdown ### Seleccione la cantidad de valores a procesar y el valor de la gravedad a utilizar
cantidad_valores = 50000 #@param {type: "number"}
gravedad = 9.8 #@param {type: "number"}
#@markdown ---
# --------------------------------------------
try:
  from datetime import datetime
  import pycuda.driver as cuda
  import pycuda.autoinit
  from pycuda.compiler import SourceModule
  import numpy
  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos 
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  # --------------------------------------------
  # Empiezo a contar el tiempo de procesamiento total
  tiempo_inicio_total = datetime.now()

  # CPU - Inicializo los vectores altura y tiempo
  vector_altura_cpu = numpy.random.uniform(1, 100000, cantidad_valores)
  vector_altura_cpu = vector_altura_cpu.astype(numpy.float32())
  vector_tiempo_cpu = numpy.empty_like(vector_altura_cpu)
  vector_tiempo_cpu = vector_tiempo_cpu.astype(numpy.float32())

  # GPU - reservo la memoria GPU.
  vector_altura_gpu = cuda.mem_alloc(vector_altura_cpu.nbytes)
  vector_tiempo_gpu = cuda.mem_alloc(vector_tiempo_cpu.nbytes)

  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod(vector_altura_gpu, vector_altura_cpu)

  # CPU - Defino la función kernel que ejecutará en GPU.
  module = SourceModule("""
  __global__ void kernel_caida_libre(int size, float *vector_altura, float *vector_tiempo, float gravedad)
  {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if(idx < size){
      vector_tiempo[idx] = sqrt((2 * vector_altura[idx]) / gravedad);
    } 
  }
  """)
  # CPU - Genero la función kernel.
  kernel = module.get_function("kernel_caida_libre")

  # Empiezo a contar el tiempo de procesamiento de GPU
  tiempo_inicio_gpu = datetime.now()

  # Armar las dimensiones correctamente.
  dim_hilo = 256
  dim_bloque = numpy.int((cantidad_valores + dim_hilo-1) / dim_hilo )
  # GPU - Ejecuta el kernel.
  kernel(numpy.int32(cantidad_valores), vector_altura_gpu, vector_tiempo_gpu, numpy.float32(gravedad), block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )
  
  # Finalizo el conteo del tiempo de procesamiento de GPU
  tiempo_fin_gpu = datetime.now()

  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh(vector_tiempo_cpu, vector_tiempo_gpu)

  # Finalizo el conteo del tiempo total de procesamiento
  tiempo_fin_total = datetime.now()

  # Muestro los valores calculados
  """
  print("Calculos con gravedad: ", gravedad, "m/s2")
  for i in range(0, cantidad_valores):
    print("Altura: ", vector_altura_cpu[i], "m - Tiempo: ", vector_tiempo_cpu[i], "s")
  """
  print("Cantidad de elementos: ", cantidad_valores)
  print("Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
  print("Tiempo Total: ", tiempo_en_ms(tiempo_fin_total - tiempo_inicio_total), "ms")
  print("Tiempo GPU: ", tiempo_en_ms(tiempo_fin_gpu - tiempo_inicio_gpu), "ms")
except Exception as excep:
  print("Error: ", excep)

---
# 4 Tabla de pasos de ejecución del programa


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura de la cantidad de valores a procesar.
CPU      |  @param                | Lectura del valor de la gravedad a utilizar.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  tiempo_en_ms          | Define una funcion para obtener el tiempo en milisegundos a partir de una fecha.
CPU      |  datetime.now()        | Toma el tiempo inicial.
CPU      |	numpy.random.uniform(1, 100000, cantidad_valores) |	Inicializa el vector de alturas
CPU      |	vector_tiempo = numpy.empty_like(vector_altura) |	Inicializa el vector de tiempos
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria texto_GPU a CPU memoria result.
CPU      |  print()               | Informo los resultados.



---
# 5 Conclusiones
### Breve repaso
En este ejercicio se realiza el calculo del tiempo que tarda un objeto en caer desde una altura determinada, usando las ecuaciones de "Caida Libre".

### Lecciones aprendidas
Como conclusion podemos observar que al realizar el calculo en paralelo de cada valor de altura del vector, usando threads del GPU, dicho calculo se realiza mucho mas rapido que si se hiciera secuencialmente. Tambien debe destacarse que la inicializacion toma bastante tiempo, por lo cual es conveniente utilizar este tipo de procesamiento cuando la cantidad de valores a calcular son bastantes.

### Sugerencias para continuar
El ejercicio se podria continuar agregando opciones para calcular tanto el tiempo, como otras variables ya sea, el valor de la gravedad o la altura.

---
# 6 Bibliografia

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

[4] Repositorio de PyCUDA: [WEB](https://pypi.python.org/pypi/pycuda)

[5] Ecuacion de Caida Libre: [Ecuaciones Caida Libre](https://es.wikipedia.org/wiki/Ecuaciones_para_un_cuerpo_en_ca%C3%ADda_libre#:~:text=Generalmente%2C%20en%20la%20atm%C3%B3sfera%20de,a%20la%20resistencia%20del%20aire)